In [4]:
import copy
import math
import random
import sys
from distutils.util import strtobool
from functools import lru_cache
import numpy as np
import torch
import torch.nn as nn
import torchaudio
import yaml
from torch.nn.utils.rnn import pad_sequence
import s3prl.optimizers
from ..baseline.extracter import get_extracter
from ..baseline.preprocessor import get_preprocessor
#from s3prl_master.s3prl.upstream.baseline.preprocessor import get_preprocessor
#from .model import TransformerConfig, TransformerModel, TransformerSpecPredictionHead

path='/home/negiryosuke/refine/s3prl/result/pretrain/George_0724_session1_30s_split_CommonAve_RegionLabel/states-300000.ckpt'
load_weights = torch.load(path)
#print(load_weights['Transformer'])
"""
options:{'load_pretrain': 'True', 'no_grad': 'False', 'dropout': 'default', 'spec_aug': 'False', 
 'spec_aug_prev': 'True', 'output_hidden_states': 'True', 'permute_input': 'False', 
 'ckpt_file': '/home/negiryosuke/refine/s3prl/result/pretrain/TERA_2/states-450000.ckpt', 'select_layer': -1}
load_weights['Config'] {'runner': {'n_epochs': -1, 'total_steps': 700000, 'gradient_clipping': 5.0, 'gradient_accumulate_steps': 4, 'log_step': 100000, 'save_step': 100000, 'max_keep': 10, 'fp16': False}, 'optimizer': {'name': 'AdamW_with_schedule', 'lr': 0.0002, 'warmup_proportion': 0.07}, 'pretrain_expert': {'datarc': {'num_workers': 8, 'train_batch_size': 8, 'max_timestep': -25, 'libri_root': '/home/negiryosuke/refine/s3prl/ECoG_data/george_20120724_session1_CommonAve_ReginLabel/', 'file_path': '/home/negiryosuke/refine/s3prl/ECoG_data/george_20120724_session1_CommonAve_ReginLabel/E_data_len_for_bucket', 'sets': ['train']}}}
load_weights['Upstream_Config'] {'transformer': {'input_dim': -1, 'hidden_size': 768, 'num_hidden_layers': 3, 'num_attention_heads': 12, 'intermediate_size': 3072, 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'attention_probs_dropout_prob': 0.1, 'initializer_range': 0.02, 'layer_norm_eps': 1e-12, 'share_layer': False, 'pre_layer_norm': False}, 'task': {'loss': 'L1', 'sequence_length': 1500, 'position_encoding_size': 768, 'mask_proportion': 0.15, 'mask_consecutive_min': 7, 'mask_consecutive_max': 7, 'mask_allow_overlap': True, 'mask_bucket_ratio': 1.5, 'mask_frequency': 0.2, 'noise_proportion': 0.0}, 'audio': {'target_level': -25, 'win_ms': 25, 'hop_ms': 10, 'n_freq': 201, 'n_mels': 80, 'n_mfcc': 13, 'input': {'feat_type': 'mel', 'channel': 0, 'log': True, 'delta': 0, 'cmvn': True}, 'target': {'feat_type': 'mel', 'channel': 0, 'log': True, 'delta': 0, 'cmvn': True}}}
self_config['audio'] {'target_level': -25, 'win_ms': 25, 'hop_ms': 10, 'n_freq': 201, 'n_mels': 80, 'n_mfcc': 13, 'input': {'feat_type': 'mel', 'channel': 0, 'log': True, 'delta': 0, 'cmvn': True}, 'target': {'feat_type': 'mel', 'channel': 0, 'log': True, 'delta': 0, 'cmvn': True}}
"""
self_config=load_weights['Upstream_Config']
target_level=25

def normalize_wav_decibel(wav):
    rms=wav.pow(2).mean().pow(0.5)
    scalar=(10**(target_level/20))/(rms+1e-10)
    wav=wav*scalar
    return wav

def preprocess(x):
    x=[normalize_wav_decibel(x_i) for x_i in x]
    x_lens=[len(x_) for x_ in x]
    x=pad_sequence(x,batch_first = True)
    x=x.unsqueeze(1)
    extracter,_,_ = get_preprocessor(self_config['audio'])
    x=extracter(x,wav_len=x_lens)[0]
    return x

ImportError: attempted relative import with no known parent package